In [15]:
import requests
import zipfile
import io
import os
from dotenv import load_dotenv

load_dotenv()

def get_and_extract_document(api_key: str, rcept_no: str, extract_path: str):
    """
    공시서류원본파일 다운로드 & CP949 인코딩으로 압축 해제
    
    :param api_key: 발급받은 API 인증키 (40자리)
    :param rcept_no: 접수번호 (14자리)
    :param extract_path: 압축 풀 경로
    """
    url = "https://opendart.fss.or.kr/api/document.xml"
    params = {
        "crtfc_key": api_key,
        "rcept_no": rcept_no
    }

    # 디렉토리가 없으면 생성
    os.makedirs(extract_path, exist_ok=True)

    # ZIP 다운로드
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"HTTP 오류: {response.status_code}")

    # 메모리에서 ZIP 열기
    with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        for info in zf.infolist():
            # 파일명 CP949 복원
            filename = info.filename.encode('cp437').decode('cp949')
            # 파일 내용 읽기
            with zf.open(info) as source, open(f"{extract_path}/{filename}", "wb") as target:
                target.write(source.read())
    print(f"압축이 해제되었습니다: {extract_path}")

# 사용 예시
# get_and_extract_document("발급받은_API_KEY", "20250112000001", "./docs")


In [16]:
DART_API_KEY = os.getenv("DART_API_KEY")
get_and_extract_document(DART_API_KEY, "20190401004781", "./docs")

압축이 해제되었습니다: ./docs
